In [9]:
from datetime import datetime
import sys
import time
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import html


api_url = "https://climatekg.semanticclimate.net/api.php"

def fetch_mediawiki_data(api_url, params):
    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from MediaWiki API: {e}")
        return None

def extract_items(data):
    items = []
    results = data.get("query", {}).get("results", {})
    
    for item_id, item_data in results.items():
        #print(item_data)
        subcats = item_data["printouts"].get("subcategoryOf", [None])
        subcat = ""
        if len(subcats) > 0: 
            subcat = subcats[0] 
        definition = ""
        if item_data["printouts"].get("definition", [{}]):
            definition = item_data["printouts"].get("definition", [{}])[0].get("Text", {}).get("item", [None])[0]
        item = {
            "id": item_id,
            "term": item_data["printouts"].get("term", [None])[0],
            "definition": definition,
            "subCategoryOf": subcat
        }
        items.append(item)
    
    return items


def generate_markdown_table(items):
    markdown = '\n| ID | Begriff | Definition | Unterkategorie von |\n'
    markdown += '|---|---|---|---|\n'

    for item in items: 
        markdown += f'|{item["id"]}'
        markdown += f'|{item["term"]}'
        markdown += f'|{item["definition"]}'
        markdown += f'|{item["subCategoryOf"]}'
        markdown += '|\n'
    
    return markdown


def get_glossary():
    params = {
        'action': 'ask',
        'query': '[[Item:+]] [[has subobject::+]] [[P1::Term]] |?Wikibase description=definition |?P3=term |?P10.P3=subcategoryOf',
        'format': 'json'
    }

    response = fetch_mediawiki_data(api_url, params)
    items = extract_items(response)
    if items:
        # Generate the markdown for the table
        markdown_table = generate_markdown_table(items)
        print(markdown_table)





In [10]:
get_glossary()


| ID | Begriff | Definition | Unterkategorie von |
|---|---|---|---|
|Item:Q10|low confidence|Each finding is grounded in an evaluation of underlying evidence and agreement. The IPCC calibrated language uses five qualifiers to express a level of confidence (very low, low, medium, high and very high )|qualifiers|
|Item:Q100|baseline/reference|The reference point or period against which changes are measured, providing a baseline for analysis.|terms|
|Item:Q1000|AOGCM|Atmosphere-Ocean General Circulation Model|acronyms|
|Item:Q1001|AOSIS|Alliance of Small Island States|acronyms|
|Item:Q1002|APEC|Asia-Pacific Economic Cooperation|acronyms|
|Item:Q1003|APP|Agricultural Adaptation and Perception|acronyms|
|Item:Q1004|APRA|Australian Prudential Regulation Authority|acronyms|
|Item:Q1005|AQ|air quality|acronyms|
|Item:Q1006|AR4|Fourth Assessment Report of the Intergovernmental Panel on Climate Change|acronyms|
|Item:Q1007|AR5|Fifth Assessment Report of the Intergovernmental Panel on Climate C